In [1]:
import cvxpy as cp
import numpy as np

https://www.cvxpy.org/

#### P1
$$ Max: \sum_{t=1}^T(5x_{1t} ​+10 x_{2t}​+(10−x_{1t}​−2x_{2t}​)+2(10−3x_{2t}​)+3(10−x_{1​t}))  \\
st: 
x_{1t} + 2x_{2t} \leq 10 \\
3x_{2t} \leq 10 \\
x_{1t} \leq 10 \\
5x_{1​t}+10x_{2t}​​​+(10-x_{1t}-2x_{2t})+2(10−3x_{2t}​)+3(10−x_{1t}​) \geq 70\\
x_1, x_2 \geq 0$$

In [2]:
T = 1
x1 = cp.Variable(T, nonneg=True)
x2 = cp.Variable(T, nonneg=True)
objective = cp.Maximize(cp.sum(5*x1 + 10*x2 + (10 - x1 - 2*x2) + 2*(10 - 3*x2) + 3*(10 - x1)))

constraints = [
    x1 + 2*x2 <= 10,
    3*x2 <= 10,
    x1 <= 10,
    5*x1 + 10*x2 + (10 - x1 - 2*x2) + 2*(10 - 3*x2) + 3*(10 - x1) >= 70
]

problem = cp.Problem(objective, constraints)
problem.solve()

if problem.status == cp.OPTIMAL:
    print("Optimal values:")
    print("Total points earned:", problem.value)
    print("Amount of tools produced:", x1.value)
    print("Amount of housing produced:", x2.value)
    print("Amount of wood left:", 10-(x1.value + 2*x2.value))
    print("Amount of stone left:", 10-(3*x2.value))
    print("Amount of iron left:", 10-x1.value)

Optimal values:
Total points earned: 70.00000000180277
Amount of tools produced: [5.77139996]
Amount of housing produced: [2.11430002]
Amount of wood left: [-1.80276949e-09]
Amount of stone left: [3.65709994]
Amount of iron left: [4.22860004]


/Users/clarkhu/opt/anaconda3/envs/dl/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


### P2
$$ max: 2x_1 + x_2  + 2x_3 + 5x_4 + 10x_5  \\
st: 
x_1 + 3x_2 + 2x_3 + x_4 + 2(x_5) \leq 100 \\
x_1 - x_4 - 2x_5 \geq 0 \\
x_2 - 3x_5 \geq 0 \\
x_3 - x_1 \geq 0 \\
x_3 \leq 10 \\
x_1, x_2, x_3, x_4,x_5 \geq 0$$
$$
\text{variables} = \begin{bmatrix} x_1 = wood \\ x_2 = stone \\ x_3 = iron \\ x_4 = tools \\ x_5 = housing \end{bmatrix}
$$

In [3]:
T = 1 
x1 = cp.Variable(T, nonneg=True)
x2 = cp.Variable(T, nonneg=True)
x3 = cp.Variable(T, nonneg=True)
x4 = cp.Variable(T, nonneg=True)
x5 = cp.Variable(T, nonneg=True)

objective = cp.Maximize(cp.sum(2*x1 + 1*x2 + 2*x3 + 5*x4 + 10*x5))
constraints = [
    x1 + 3*x2 + 2*x3 + x4 + 2*x5 <= 100,
    x1 - x4 - 2*x5 >=0,
    x2 - 3*x5 >=0,
    x3 - x1 >=0,
    x3 <= 10
]

problem = cp.Problem(objective, constraints)
problem.solve()
if problem.status == cp.OPTIMAL:
    print("Optimal values:",problem.value)
    print("Wood used:", x1.value)
    print("Stone used:", x2.value)
    print("Iron extracted:", x3.value)
    print("Tools produced:", x4.value)
    print("Housing produced:", x5.value)

Optimal values: 109.99999999985614
Wood used: [10.]
Stone used: [20.]
Iron extracted: [10.]
Tools produced: [3.63363684]
Housing produced: [3.18318158]


#### P3

$$\max \sum_{t} \left(1 \cdot (W_t - S_{t+1}^W) + 2 \cdot (S_t - S_{t+1}^S) + 3 \cdot (I_t - S_{t+1}^I) + 5 \cdot (T_t + T_{t+1}) + 10 \cdot H_t\right) $$

**St:**

1. **Resource Conservation:**
   $$
   P^W_t + S^W_t - C^W_t \geq 0 \\
   P^S_t + S^S_t - C^S_t \geq 0 \\
   P^I_t + S^I_t - C^I_t \geq 0 \\
   P^T_t + S^T_t - C^T_t \geq 0 \\ $$
2. **Stockpile Capacity:**
   $$
   S^W_t + S^S_t + S^I_t + S^T_t \leq 30
   $$
3. **Initial Stockpile:**
   $$
   S^W_1 = 5, \quad S^S_1 = 5, \quad S^I_1 = 5, \quad S^T_1 = 5
   $$
4. **Stockpile Limitation:**
   $$
   S^W_{t+1} \leq W_t \\
   S^S_{t+1} \leq S_t \\
   S^I_{t+1} \leq I_t \\
   S^T_{t+1} \leq T_t \\
   $$
5. **Mining Constraint:**
   $$
   S_1 \leq 5 \\ 
   \quad S_{t+1} \leq 5 + 5 \cdot M_t
   $$
6. **Labour Constraint:**
   $$
   L_1 \leq 50 \\ 
   \quad L_{t+1} \leq 50 + 10 \cdot H_t
   $$

**Variables:**

$$
\text{Resources} = \begin{bmatrix} W \\ S \\ I \\ T \\ M \\ H \\ SH \end{bmatrix}, \quad \text{Production} = \begin{bmatrix} P^W \\ P^S \\ P^I \\ P^T \\ C^W \\ C^S \\ C^I \\ C^T \\ L \end{bmatrix}
$$


In [4]:
num_resources = 4
num_products = 5

stockpile = cp.Variable((num_resources, num_products), nonneg=True)
production = cp.Variable((6, num_products), nonneg=True)
points = cp.Variable((num_products, num_products), nonneg=True)

costs = np.array([
    [0, 0, 0, 0.1, 1], 
    [0, 0, 0, 0.1, 3], 
    [0, 0, 0, 0.1, 2], 
    [1, 0, 1, 0.1, 1], 
    [2, 3, 0, 0.2, 2], 
    [0, 0, 0, 0.2, 1],
])

scores = np.array([1, 2, 3, 5, 10])

resource_cost = (costs.T @ production)[:num_resources]
labour_cost = (costs.T @ production)[num_resources]

resource_production = production[:num_resources]
housing, mining = production[num_resources:]

iron_production = production[2]

resource_points = points[:num_resources]
housing_points = points[num_resources]

constraints = [
    points <= production[:num_products],
    resource_cost <= stockpile,
    stockpile.sum(axis=0) <= 30,
    stockpile.T[0] == 5,
]

for i in range(num_resources):
    constraints.append(stockpile.T[i + 1] == resource_production.T[i] - resource_points.T[i])
for i in range(1, num_products):
    constraints.append(labour_cost[i] <= 50 + 10 * (housing - housing_points)[:i].cumsum())
    constraints.append(iron_production[i] <= 5 + mining[:i].cumsum())
    
problem = cp.Problem(cp.Maximize((scores @ points).sum()), constraints)
problem.solve()

print(f"Optimal value: {problem.value:.2f}")
print(f"Optimal stockpile:\n{stockpile.value}")
print(f"Optimal production:\n{production.value}")

Optimal value: 297.07
Optimal stockpile:
[[5.00000000e+00 2.54715809e+00 2.38796077e+00 2.23871325e+00
  5.73644571e-07]
 [5.00000000e+00 3.09032883e-09 2.34188871e-08 3.43890946e-08
  4.65228664e-07]
 [5.00000000e+00 2.54715809e+00 2.38796075e+00 2.23871323e+00
  2.62438524e-07]
 [5.00000000e+00 2.56772108e+00 2.54715809e+00 2.38796075e+00
  2.23871322e+00]]
Optimal production:
[[2.54715809e+00 2.38796077e+00 2.23871326e+00 6.36305871e-07
  4.43842350e-07]
 [5.50399808e-09 1.57420919e+01 1.58449069e+01 1.66408936e+01
  1.73871312e+01]
 [4.24528419e+01 5.00000001e+00 5.00000001e+00 5.00000001e+00
  5.00000001e+00]
 [2.56772108e+00 2.54715809e+00 2.38796075e+00 2.23871323e+00
  2.62290937e-07]
 [1.21613946e+00 9.10578033e-10 7.74967090e-09 1.14186249e-08
  1.55045619e-07]
 [7.36804245e-09 3.67153919e-09 1.21018759e-09 9.21014192e-10
  7.60981547e-10]]
